In [3]:
# download data

import os
import urllib.request

urls = ["http://cs.joensuu.fi/sipu/datasets/jain.txt", "http://cs.joensuu.fi/sipu/datasets/flame.txt"]

for url in urls:
  filename = url.split("/")[-1]

  # rename to csv
  filename = filename.replace(".txt", ".csv")

  # check if file exists
  if os.path.isfile(filename):
    print("File already exists: " + filename)
    continue
  
  # download file
  urllib.request.urlretrieve(url, filename)

  print("Downloaded: " + filename)





File already exists: jain.csv
File already exists: flame.csv
